##### Mounting the Blob Storage Container by Active Directory App Credentials
To make mounting by this method, Soft Delete should be disabled for storage account. Otherwise the mounting script can give error. Go to Storage Account > Data Protection
To get ClientId and TenantId, go to App Registrations and create a service app. To get a Client Secret go to Certificates & secrets section and create a Client Secret code. Then go to Storage Account IAM section and assign role. 

In [ ]:
storage_account_name = 'blobsaless'
container_name = "amazon-vendors-api-all"
client_id = '7db61c6.......33'
tenant_id = 'ea3ed546.....6-69fad2403d4c'
client_secret = '2X48Q~Fp......Jl1jXT4AcVY'


configs = {
            "fs.azure.account.auth.type": "OAuth",
            "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
            "fs.azure.account.oauth2.client.id": f"{client_id}",
            "fs.azure.account.oauth2.client.secret": f"{client_secret}",
            "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
          }


try:
    dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")
except:
    pass
    
try:
    dbutils.fs.mount(
                     source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
                     mount_point = f"/mnt/{storage_account_name}/{container_name}",
                     extra_configs = configs
                    )
except:
    pass

In [0]:
%fs mounts

In [0]:
# To check the files in mounting
dbutils.fs.ls('/mnt/blobsaless/amazon-vendors-api-all')

##### Mounting the Blob Storage Container by SAS Token
sas token can be taken from the container settings > shared access tokens.
Permissions should be given Read, Write, Delete ..(sp=racwdli&st) while generating sas token. Give expire date range widely so that dont get auth error.

In [0]:
containerName = "amazon-vendors-api-all"
storageAccountName = "blobsaless"
sas = "?sp=racwdli&st=20229%2F7.........................PtdFkx0%3D"
config = "fs.azure.sas." + containerName+ "." + storageAccountName + ".blob.core.windows.net"

try:
    dbutils.fs.unmount("/mnt/vendor-data-api-all")
except:
    pass
    
try:
    dbutils.fs.mount(
      source = f"wasbs://{containerName}@{storageAccountName}.blob.core.windows.net",
      mount_point = "/mnt/vendor-data-api-all",
      extra_configs = {config : sas})
except:
    pass